<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Interactive Dashboard with Ploty Dash**


We will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in real-time.

This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart. We will be guided to build this dashboard application via the following tasks:

*   TASK 1: Add a Launch Site Drop-down Input Component and Range Slider to Select Payload
*   TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
*   TASK 3: Add a callback function to render the success-payload-scatter-chart scatter plot


Let's first import required Python packages:

In [1]:
!pip install wget
!pip install dash

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=e303b1f6fe44d9b1a3c4a09d40b61bf51039e0fe925eb7bb136962e1858e08fc
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


In [15]:
import wget
import pandas as pd
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

## **TASK 1: Add a Launch Site Drop-down Input Component**

We have four different launch sites and we would like to first see which one has the largest success count. Then, we would like to select one specific site and check its detailed success rate (class=0 vs. class=1).

As such, we will need a dropdown menu to let us select different launch sites.

In [16]:
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')
spacex_df=pd.read_csv(spacex_csv_file)
spacex_df.head(5)

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


Preparing Dropdown Options

In [18]:
launch_sites = spacex_df['Launch Site'].unique().tolist()
dropdown_options = [{'label': 'All Sites', 'value': 'ALL'}] + [
    {'label': site, 'value': site} for site in launch_sites]

Create the Dash App

In [19]:
app = Dash(__name__)

App Layout

In [20]:
app.layout = html.Div([
    html.H1("SpaceX Launch Dashboard", style={'textAlign': 'center'}),

    dcc.Dropdown(
        id='site-dropdown',
        options=dropdown_options,
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    html.Br(),

    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()],
        marks={i: str(i) for i in range(0, 10001, 1000)},
        tooltip={"placement": "bottom", "always_visible": True}
    ),
    html.Br(),

    dcc.Graph(id='success-pie-chart'),
    html.Br(),

    dcc.Graph(id='success-payload-scatter-chart'),
], style={'width': '70%', 'margin': 'auto'})

## **TASK 2: Add a callback function to render Success-Pie-Chart based on selected site dropdown**

In [21]:
@app.callback(
    Output('success-pie-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_pie_chart(selected_site, payload_range):
    min_payload, max_payload = payload_range
    df_filtered = spacex_df[(spacex_df['Payload Mass (kg)'] >= min_payload) &
                            (spacex_df['Payload Mass (kg)'] <= max_payload)]

    if selected_site == 'ALL':
        df_grouped = df_filtered.groupby('Launch Site')['class'].sum().reset_index()
        fig = px.pie(df_grouped,
                     names='Launch Site',
                     values='class',
                     title='Total Success Launches by Site (Filtered by Payload)')
    else:
        df_filtered = df_filtered[df_filtered['Launch Site'] == selected_site]
        class_counts = df_filtered['class'].value_counts().reset_index()
        class_counts.columns = ['class', 'count']
        class_counts['class'] = class_counts['class'].replace({1: 'Success', 0: 'Failure'})
        fig = px.pie(class_counts,
                     names='class',
                     values='count',
                     title=f'Success vs Failure for {selected_site} (Filtered by Payload)')
    return fig

## **TASK 3: Add a callback function to render the success-payload-scatter-chart scatter plot**

In [22]:
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_chart(selected_site, payload_range):
    min_payload, max_payload = payload_range
    df_filtered = spacex_df[(spacex_df['Payload Mass (kg)'] >= min_payload) &
                            (spacex_df['Payload Mass (kg)'] <= max_payload)]

    if selected_site == 'ALL':
        fig = px.scatter(df_filtered,
                         x='Payload Mass (kg)',
                         y='class',
                         color='Booster Version Category',
                         title='Payload vs Outcome for All Sites')
    else:
        df_filtered = df_filtered[df_filtered['Launch Site'] == selected_site]
        fig = px.scatter(df_filtered,
                         x='Payload Mass (kg)',
                         y='class',
                         color='Booster Version Category',
                         title=f'Payload vs Outcome for {selected_site}')
    return fig

Run the App

In [23]:
# Run the app
if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>

1.   Which site has the largest successful launches?


	KSC LC-39A with 41.7%
2.   Which site has the highest launch success rate?


	KSC LC-39A with 76.9%
3.   Which payload range(s) has the highest launch success rate?


	2000kg to 4000kg
4.   Which payload range(s) has the lowest launch success rate?


	0kg to 2000kg
5.   Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?


	FT


Copyright © 2021 IBM Corporation. All rights reserved.